In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)
import sklearn as skl
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

In [127]:
# pre process
iris = sns.load_dataset("iris");
iris.dropna(axis=0, how='any', inplace=True);
print(iris.head(5));
target_df = pd.get_dummies(iris['species']);
iris  = iris.drop(['species'], axis=1);
scaler = MinMaxScaler()
#iris[iris.columns] = scaler.fit_transform(iris[iris.columns])
df = pd.concat([iris, target_df], axis=1)
#print(df.head(5))
train = df.sample(frac=0.8,random_state=10)
test = df.drop(train.index)
print("Number of training saplmes: ", len(train))
outclass = ['setosa', 'versicolor', 'verginica']  

   sepal_length  sepal_width  petal_length  petal_width species
0           5.1          3.5           1.4          0.2  setosa
1           4.9          3.0           1.4          0.2  setosa
2           4.7          3.2           1.3          0.2  setosa
3           4.6          3.1           1.5          0.2  setosa
4           5.0          3.6           1.4          0.2  setosa
Number of training saplmes:  120


In [142]:
class parameters():
    with tf.variable_scope('weights_biases'):
        def __init__(self, n_features, n_hidden_layers, n_hidden_neurons, n_output):
            self.n_features = n_features;
            self.n_hidden_layers = n_hidden_layers;
            self.n_hidden_neurons = n_hidden_neurons;
            self.n_output = n_output;
            self.bias = []
            self.weights = [];
            assert (len(n_hidden_neurons) == n_hidden_layers), \
                "Length of #nerons list must be equal to #hidden layers ";
            for i in np.arange(self.n_hidden_layers+1):
                if i==0:
                    self.weights.append(tf.Variable(np.random.randn(n_features, \
                            n_hidden_neurons[0]), dtype=tf.float32))
                elif i==self.n_hidden_layers:
                     self.weights.append(tf.Variable(np.random.randn(n_hidden_neurons[i-1],n_output), \
                            dtype=tf.float32))
                else:
                     self.weights.append(tf.Variable(np.random.randn(n_hidden_neurons[i-1], \
                            n_hidden_neurons[i]), dtype=tf.float32))
                    
            for i in np.arange(self.n_hidden_layers+1):
                if i < self.n_hidden_layers:
                    self.bias.append(tf.Variable(tf.zeros((1, n_hidden_neurons[i])), dtype=tf.float32))
                else:
                    self.bias.append(tf.Variable(tf.zeros((1,n_output)), dtype=tf.float32))


""" Build the model """
param = parameters(4,1,[4],3);
X = tf.placeholder(tf.float32, shape=(None, param.n_features), name='X');
y = tf.placeholder(tf.int32, shape=(None, param.n_output), name='y');
hidden_one = tf.nn.relu(tf.add(tf.matmul(X, param.weights[0]), param.bias[0]));
output_layer = tf.add(tf.matmul(hidden_one, param.weights[1]), param.bias[1]);
y_ = tf.nn.softmax(output_layer)

""" Optimize the model """
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output_layer)
loss = tf.reduce_mean(xentropy, name="loss")
lrate = 0.1;
optimizer = tf.train.GradientDescentOptimizer(learning_rate = lrate)
training_oprn = optimizer.minimize(loss)

"""check accuracy"""
correct_prediction = tf.equal(tf.argmax(y_,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

init = tf.global_variables_initializer()
saver = tf.train.Saver()
n_epochs = 101
batch_size = 12
init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
    for epoch in range(n_epochs):
        for iteration in range(df.shape[0] // batch_size):
            X_batch, y_batch = train.iloc[iteration*batch_size:iteration*batch_size+batch_size, 0:4], \
                               train.iloc[iteration*batch_size:iteration*batch_size+batch_size, 4:7]
            sess.run(training_oprn, feed_dict={X: X_batch, y: y_batch})
        X_batch, y_batch = train.iloc[:, 0:4], train.iloc[:,4:7]     
        if epoch % 100 == 0:
            print("epoch: {0:4d}, Train accuracy: {1:1.2f}".format(epoch, sess.run(accuracy,feed_dict={X: X_batch, y: y_batch})))
    X_batch, y_batch = test.iloc[:, 0:4], test.iloc[:,4:7] 
    print("test accuracy: {0:1.2f}".format(sess.run(accuracy,feed_dict={X: X_batch, y: y_batch})))
    saver.save(sess, "./iris_model.ckpt")   

epoch:    0, Train accuracy: 0.68
epoch:  100, Train accuracy: 0.97
test accuracy: 0.97


In [135]:
with tf.Session() as sess:
  # Restore variables from disk.
    saver.restore(sess, "./iris_model.ckpt")    
    X1 = pd.DataFrame(data={"sepal_length": [5.0], "sepal_width": [1.0], "petal_length": [1.4], "petal_width": [0.2]})
    # enforce the order of columns otherwise pandas can change the order 
    X1 = X1[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']]
    print("Given feature values: ")
    print(X1);    
    #X1[X1.columns] = scaler.fit_transform(X1[X1.columns]);
    i = sess.run(tf.cast(tf.argmax(y_,1), tf.int8),feed_dict={X: X1})
    outputclass = [outclass[ii] for ii in np.arange(len(i))];
    print("corresponding predicted class: ", outputclass);

INFO:tensorflow:Restoring parameters from ./iris_model.ckpt
Given feature values: 
   sepal_length  sepal_width  petal_length  petal_width
0           5.0          1.0           1.4          0.2
corresponding predicted class:  ['setosa']
